In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold, train_test_split
import joblib

**LOADING THE DATA**

In [ ]:
dataBin = pd.read_csv('train.csv')

In [ ]:
dataBin.head(5)

**CREATING THE TEST SAMPLES FOR TRAINING**

Because the dataset is not balanced, I thought of only taking 10000 entries from each rating

In [ ]:
toTrain = pd.DataFrame()

In [ ]:
toTrain = dataBin[dataBin['rating'] == 1].sample(n=10000, replace=False, random_state=1)
for i in range(2,6):
    toTrain = pd.concat([toTrain, dataBin[dataBin['rating'] == i].sample(n=10000, replace=False, random_state=1)], axis=0)

In [ ]:
toTrain

**CREATING THE VECTORIZER THAT WILL PARSE THE WORDS INTO PROCESSIBLE VECTORS**

In [ ]:
vectorizer = CountVectorizer()

**FUNCTIONS THAT WILL TRAIN THE MODEL**

I made spam detector previously, and shimity shems I go blind here

I'll explain what each functions below do:
- met_MachineModelScore => this scores the model like in kaggle
- met_MachineModel => the one that trains the model
- met_kFolding_MachineTest => what kfolding means is that the dataset will be folded into N groups, and the test database will be tested to these five groups individually. model will be trained N times.


In [ ]:
def met_MachineModelScore(model, X_train, y_train, X_test, y_test):
            model.fit(X_train, y_train)
            return model.score(X_test, y_test)

In [ ]:
def met_MachineModel(model, X_train, y_train):
        x = model.fit(X_train, y_train)
        return x

In [ ]:
def met_kFolding_MachineTest(X_set, y_set, model = [], k_splits = 4, shuf = False ,ret_val = 'Scores'):

        folder = KFold(n_splits=k_splits, shuffle=shuf)

        model_Score = dict()
        model_Machine = dict()

        for mod in model:
            model_Score[mod.__class__.__name__] = []
            model_Machine[mod.__class__.__name__] = mod

        for train_i, test_i in folder.split(X_set, y_set):
            X_train, X_test, y_train, y_test = X_set[train_i], X_set[test_i], y_set[train_i], y_set[test_i]

            print("Train Size: ", len(y_train), " Test Size:",len(y_test))

            for mod in model:

                model_Machine[mod.__class__.__name__] = (met_MachineModel(model_Machine[mod.__class__.__name__], X_train, y_train))
                model_Score[mod.__class__.__name__].append(met_MachineModelScore(model_Machine[mod.__class__.__name__], X_train, y_train, X_test, y_test))

        print()

        if (ret_val == 'Scores'):
            return model_Score
        elif (ret_val == 'Models'):
            return model_Machine
        elif (ret_val == 'All'):
            return model_Score, model_Machine

Before passing the test database into the trainer, you must vectoriz it. Fit_transform does everything

In [ ]:
X_train_cv = vectorizer.fit_transform(dataBin.review)

In [ ]:
mod_Score, mod_Data = met_kFolding_MachineTest(X_train_cv, dataBin.rating, [MultinomialNB()], k_splits = 5, shuf=True, ret_val = 'All')

for key in mod_Score:
    print(key)
    print(mod_Score[key])

print()

for keys in mod_Data:
    print(keys," : ", mod_Data[keys].score(X_train_cv, dataBin.rating))

**PREDICTING THE CSV TO BE PASSED**

In [ ]:
toPredict = pd.read_csv('test.csv')

In [ ]:
toPredict

In [ ]:
toPredictVector = vectorizer.transform(toPredict.review)

In [ ]:
toPredict['rating'] = model.predict(toPredictVector)

In [ ]:
toPredict

In [ ]:
toPredict = toPredict.drop(columns=['review'])
toPredict

In [ ]:
toPredict.to_csv("toPass.csv", index=False)